In [ ]:
import torch
import numpy as np
from lieflow.groups import Rn
from lieflow.models import (
    FlowFieldProductGroup,
    LogarithmicDistance
)
import matplotlib.pyplot as plt
from matplotlib.animation import FFMpegWriter
import matplotlib.colors as mcolors
import matplotlib.gridspec as gridspec
from matplotlib.cm import ScalarMappable
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Computer Modern"],
    "font.size": 10.0,
    "text.latex.preamble": r"\usepackage{lmodern} \usepackage{amssymb} \usepackage{amsmath}"
})
from tqdm import tqdm

In [ ]:
r2 = Rn(2)
M = 10

In [ ]:
generate_videos = True

# Distributions

In [ ]:
def generate_normals(N, m):
    return torch.Tensor(np.random.randn(N, m, 2))

def generate_uniforms_on_circle(N, m, ε=0.04):
    angles_unshifted = np.random.rand(N) * 2 * np.pi
    dθ = 2 * np.pi/m
    angles = torch.Tensor(np.arange(m)[..., None] * dθ + angles_unshifted)[..., None]
    xs = torch.cat((torch.cos(angles), torch.sin(angles)), dim=-1).transpose(0, 1)
    return xs + ε * torch.randn(xs.shape)


In [ ]:
t = generate_uniforms_on_circle(3, M)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5))
for x in t:
    ax.scatter(x[..., 0], x[..., 1])
ax.set_xlim(-3, 3)
ax.set_ylim(-3, 3);

In [ ]:
tests = ("normals_to_circle",)

In [ ]:
def data_generator(test, ε=0.04):
    match test:
        case "normals_to_circle":
            generate_x_0 = lambda n: generate_normals(n, M)
            generate_x_1 = lambda n: generate_uniforms_on_circle(n, M, ε=ε)
    return generate_x_0, generate_x_1

# Models

## Training

In [ ]:
EPSILON = 0.03
N = 2**12
BATCH_SIZE = 2**8
EPOCHS = len(tests) * [100]
WEIGHT_DECAY = 0.
LEARNING_RATE = 5e-4
H = 64
L = 4
device = "cuda"

In [ ]:
def train_model(x_0s, x_1s, epochs, batch_size=BATCH_SIZE, learning_rate=LEARNING_RATE, weight_decay=WEIGHT_DECAY):
    train_loader = torch.utils.data.DataLoader(
        torch.utils.data.TensorDataset(x_0s, x_1s), batch_size=batch_size, shuffle=True
    )

    model_FM = FlowFieldProductGroup(r2, embed_dim=H, L=L).to(device)
    optimizer_FM = torch.optim.Adam(model_FM.parameters(), learning_rate, weight_decay=weight_decay)
    loss = LogarithmicDistance(torch.Tensor([1., 1.]).to(device))

    losses_FM = np.zeros(epochs)
    for i in tqdm(range(epochs)):
        losses_FM[i] = model_FM.train_network(device, train_loader, optimizer_FM, loss)

    fig, ax = plt.subplots(1, 1, figsize=(5, 5))
    ax.plot(losses_FM)
    ax.set_title("Batch Loss Flow Matching")
    ax.set_xscale("log")
    ax.set_yscale("log")

    return model_FM, losses_FM

In [ ]:
models_FM = {}
losses_FM = {}
for i, test in enumerate(tests):
    print(test)
    generate_x_0, generate_x_1 = data_generator(test, ε=EPSILON)

    x_0s = generate_x_0(N)
    x_1s = generate_x_1(N)

    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    for j in range(M):
        ax[0].scatter(x_0s[:32, j, 0], x_0s[:32, j, 1])
        ax[1].scatter(x_1s[:32, j, 0], x_1s[:32, j, 1])
    ax[0].set_xlim(-3, 3)
    ax[0].set_ylim(-3, 3)
    ax[0].set_title(r"$\mathfrak{X}_0$")
    ax[1].set_xlim(-3, 3)
    ax[1].set_ylim(-3, 3)
    ax[1].set_title(r"$\mathfrak{X}_1$")

    models_FM[test], losses_FM[test] = train_model(x_0s, x_1s, epochs=EPOCHS[i], batch_size=BATCH_SIZE, learning_rate=LEARNING_RATE,
                                  weight_decay=WEIGHT_DECAY)

# Testing

In [ ]:
def create_animations(tests, models_FM, N=2**5):
    N_models = len(tests)

    x_0s_test = {}
    x_ts_test = {}
    for test in tests:
        generate_x_0, _ = data_generator(test, ε=EPSILON)
        x_0s_test[test] = generate_x_0(N).to(device)
        x_ts_test[test] = x_0s_test[test].detach().clone()
        
        models_FM[test].eval()

    t = 0
    N_steps = 120
    Δt = 1. / N_steps
    metadata = {'title': 'Flow Matching R^2', 'artist': 'Matplotlib'}
    writer = FFMpegWriter(fps=30, metadata=metadata)

    fig, ax = plt.subplots(1, N_models, figsize=(5 * N_models, 5))

    with writer.saving(fig, f"output/flow_matching_R2_n.mp4", dpi=150):
        for i, test in enumerate(tests):
            x_ts_test_plot = x_ts_test[test].to("cpu")
            for j in range(M):
                ax.scatter(
                    x_ts_test_plot[:, j, 0], x_ts_test_plot[:, j, 1]
                )
            ax.set_title(fr"$\mathfrak{{X}}_{{{t:.2f}}}$")
            ax.set_xlim(-3, 3)
            ax.set_ylim(-3, 3)
        writer.grab_frame()
        for frame in tqdm(range(N_steps)):
            t = frame * Δt
            for test in tests:
                with torch.no_grad():
                    x_ts_test[test] = models_FM[test].step(x_ts_test[test], torch.Tensor([t]).to(device), Δt)
                x_ts_test_plot = x_ts_test[test].to("cpu")
                ax.clear()
                for j in range(M):
                    ax.scatter(
                        x_ts_test_plot[:, j, 0], x_ts_test_plot[:, j, 1]
                    )
                ax.set_title(fr"$\mathfrak{{X}}_{{{t:.2f}}}$")
                ax.set_xlim(-3, 3)
                ax.set_ylim(-3, 3)
            writer.grab_frame()
        writer.grab_frame()

In [ ]:
if generate_videos:
    create_animations(tests, models_FM, N=1)

In [ ]:
# t = 0
# N_steps = 240
# Δt = 1. / N_steps
# N_show = 5
# N_skip = int(N_steps / (N_show-1))

# N_models = len(tests)
# N_samples = 32

# x_0s_test = {}
# x_1s_test = {}
# x_ts_test = {}
# for test in tests:
#     generate_x_0, generate_x_1 = data_generator(test, ε=EPSILON)
#     x_0s_test[test] = generate_x_0(N_samples).to(device)
#     x_1s_test[test] = generate_x_1(N_samples).to(device)
#     x_ts_test[test] = x_0s_test[test].detach().clone()
    
#     models_FM[test].eval()

# fig = plt.figure(figsize=(4.8, 1.6 * N_models * 3/3.1))
# gs = gridspec.GridSpec(N_models, 4, width_ratios=[1, 1, 1, 0.1], height_ratios=N_models * [1.], wspace=0.1, hspace=0.1)
# cax = fig.add_subplot(gs[:, 3])

# ax = []
# for i in range(N_models):
#     ax.append([])
#     for j in range(3):
#         a = fig.add_subplot(gs[i, j])
#         a.set_xlim(-3, 3)
#         a.set_ylim(-3, 3)
#         a.set_xticks([])
#         a.set_yticks([])
#         a.set_aspect("equal")
#         ax[i].append(a)
        
# ax[0][0].set_title(r"$\mathfrak{X}_0$")
# ax[0][1].set_title(r"$\mathfrak{X}_t$")
# ax[0][2].set_title(r"$\mathfrak{X}_1$")
# Δc = 1 / (N_show - 1)
# colors = [(j * Δc, 0.1, 1 - j * Δc) for j in range(N_show)]
# cmap = mcolors.ListedColormap(colors)
# for i, test in enumerate(tests):
#     k = 0

#     ax[i][0].scatter(
#         x_0s_test[test][:N_samples, 0], x_0s_test[test][:N_samples, 1], marker="."
#     )
#     ax[i][2].scatter(
#         x_1s_test[test][:N_samples, 0], x_1s_test[test][:N_samples, 1], marker="."
#     )

#     alpha = 1
#     N_samples_shown = N_samples
#     for j in range(N_steps+1):
#         t = j * Δt
#         if j == N_steps:
#             alpha = 1
#             N_samples_shown = N_samples
#         with torch.no_grad():
#             x_ts_test[test] = models_FM[test].step(x_ts_test[test], torch.Tensor([t]), torch.Tensor([Δt]))

#         if j % N_skip == 0:
#             x_ts_test_plot = x_ts_test[test].to("cpu")
#             im = ax[i][1].scatter(
#                 x_ts_test_plot[:N_samples_shown, 0], x_ts_test_plot[:N_samples_shown, 1],
#                 color=colors[k], marker=".", alpha=alpha
#             )
#             k += 1
#         alpha = 0.5
#         N_samples_shown = N_samples // 4
# fig.colorbar(ScalarMappable(cmap=cmap), cax=cax, ticks=np.linspace(0, 1, N_show), label="$t$");
# fig.savefig(f"output/interpolation_R2_n.pdf", bbox_inches="tight")